<div style="text-align:right;color:navy">Machine Learning & Data Analysis &nbsp;  &nbsp;  &nbsp; December 3, 2021</div>

# Machine Learning &nbsp; &nbsp;


## Lab 5 : K-Fold & Cross-validation

<br />

我們在介紹 Iris Dataset 時曾經提過，如果在相同資料集(像是 Iris)上進行 Model 的訓練與準確度的測試(例如使用上次看到的 accuracy_score)，這樣準確度的測試結果應該會很好!! 但是如果面對沒看過的資料，則可能無法預測得太好。這種情況稱為 overfitting (過擬合），為了避免這種情況，在進行（監督）機器學習，訓練模型時，不把所有的資料都拿來讓機器學習，而是取出部分的資料作為測試集（test set）。上次我們利用 scikit-learn 的 [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) 函數，將實驗資料集劃分為訓練集（training sets）和測試集（test sets）。

我們再重複上次利用 iris dataset 操作 Train_test_split 的過程 :

#### Step 0-1 : 匯入資料

In [ ]:
# 匯入 load_iris 函數
from sklearn.datasets import load_iris
# 取得資料
iris = load_iris()
# 依慣例，我們以 X 來儲存 data
X = iris.data
# 以 y 來儲存 target
y = iris.target

#### Step 0-2 : Split 資料

* 留三分之一作為測試，test_size=0.33。  
* random_state 所設定的值會影響到 X_train, X_test, y_train, y_test 的配置，所以如果要做任何比較時(e.g.準確度)，必須使用相同的參數值才客觀(e.g. random_state=25)。

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=25)

#### Step 1 :  匯入要使用的演算法 class

使用 KNeighborsClassifier (k-nearest neighbors 最近鄰)方法建立估計器模型，然後在資料集 X_train, y_train 上訓練。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


#### Step 2 :  建立一個 Model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

#### Step 3 : 使用資料來訓練 Model

In [ ]:
knn.fit(X_train, y_train)

#### Step 4 : 使用 Model 來測試 Testing data & 準確度評估

In [ ]:
y_pred = knn.predict(X_test)

我們會再討論準確度評估的方法 ，在這裡就先接受 accuracy_score 這一個評估方法。

In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

## Cross Validation  交叉驗證

但是這樣的 train_test_split 的操作也未必解決 overfitting 的問題，例如在 knn 中，我們可以透過 n_neighbors 這個 hyperparameters(超參數)的調整來得到不同的 Models，接著我們可以從其中選出在 accuracy_score 獲得分數最好的 Model，這樣也算是找到最適合這個 test set 的 Model 了。Overfitting 就可能產生了!! 我們在 MLpSet3 中處理過這個問題!!

所以除了 Training set、test set 之外，應該再準備另一部分被稱為 "validation set(驗證集)" 的資料集，模型訓練完成以後先在驗證集上對模型進行評估。 當驗證集上的評估實驗比較成功時，在未知的測試集上再進行最後的評估。然而，通過將原始資料分為3個資料集合 - 訓練集（training sets）、驗證集(validation set)和測試集（test sets），我們就大大減少了可用於模型訓練的樣本數量，而且得到的結果過於依賴對訓練集與驗證集的"隨機選擇"。

這個問題可以通過 Cross-Validation (交叉驗證，簡稱 CV）來解決。 交叉驗證仍需要測試集（test sets）做最後的模型評估，但不再需要驗證集(validation set)。 在所有CV方法中， 最基本的方法被稱之為，**K-fold CV (k-折交叉驗證)**。k-fold 交叉驗證將原始的完整訓練集劃分為 k 個較小的集合。 每一個 k "folds" 都會遵循下面的過程 :


1. 將資料分成 k 個同樣大小的子集合
2. a. 以第一個子集合做為 Testing Set，其餘 k-1 個子集合聯集為 Training Set<br />
   b. 計算 Testing Set 的準確度
3. a. 再將第二個子集合做為 Testing Set，其餘 k-1 個子集合聯集為 Training Set<br />
   b. 計算 Testing Set 的準確度
4. ... 依此方法總共做 k 次的計算
5. 使用這 k 個準確度的平均值做為這一個 Model 的準確度

[參考官網](https://scikit-learn.org/stable/modules/cross_validation.html)

<img src="../img/grid_search_cross_validation.png" width="600" />

### 不同的 split 方法

做交叉驗證(cross-validation)的第一步是：**將資料分成 k 個同樣大小的子集合**，我們今天來介紹兩個分割資料的方法，分別為 [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) 及 [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold)。


使用 sklearn.model_selection 的 k-fold :

可參考官網說明
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

下面是改自官網的例子 :

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 1, 2])
kf = KFold(n_splits=2)

# 可以顯示 n_splits(=2) 組 Training Set 跟 Testing Set 的 indeices，注意 : 是 index!!
for train_index, test_index in kf.split(X):
    print("TrainIndex:", train_index, "TestIndex:", test_index)


上面所得到的(兩組)資料是 Training Set 跟 Testing Set 的 indices，同學可以使用 data selection 的方式來看一下 Training Set 跟 Testing Set，例如 `X[train_index]`、`X[test_index]`、`y[train_index]`、`y[test_index]`，請同學試試:

In [ ]:
# 執行完上面的 for 迴圈後，TrainIndex 是 [0 1]，TestIndex 是 [2 3]
X[train_index]
# X[test_index]
# y[train_index]
# y[test_index]

這次我們來看看這兩組 Training Set 跟 Testing Set :

In [ ]:
# print 的部分，我們使用了 string format

import numpy as np
from sklearn.model_selection import KFold
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 1, 2])
kf = KFold(n_splits=2)

# 可以顯示 n_splits(=2) 組 Training Set 跟 Testing Set
n = 0
for train_index, test_index in kf.split(X):
    n += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("第 {} 組 \n Training Set: \n {} \n Testing Set: \n {}".format(n, X_train, X_test))

### Exercise 1 
請將上面的 n_splits 設定成 3 執行看看，請自己解釋看看你所看到的 output

In [ ]:
# Exercise 1 




### Exercise 2 

請同學以下面的資料為 X 及 y，設定 n_splits = 4，執行一下上面的程式，除了 X_train, X_test，也把 y_train, y_test 顯示出來試試 :

    X = np.array(range(1, 25))
    y = np.tile(np.array([0, 1]), 12) 
    
這裡的 np.tile(A, k) 是指將 array A 複製 k 次所得到的新 array

In [ ]:
# Exercise 2 




### Exercise 3

重設 target y (下面的 np.random.choice 等於是將 y 作重新隨機排列)
          
          y_new = np.random.choice(y,size=24,replace=False)
          
重複 Exercise 2 的步驟，並且將 kf.split(X) 取代為 kf.split(X, y_new)，觀察驗證集與 Exercise 2 是否有差異？

In [ ]:
# Exercise 3 

y_new = np.random.choice(y,size=24,replace=False)



####  k-fold 做 Cross-Validation 的問題

我們發覺 k-fold 資料分組的方式，和分類資料 y 沒有關聯， 基本上是依照分組個數 n_splits，由訓練組(training set)當中循序安排驗證集，不過如果是訓練分類的模型 (Classification Model)，在將資料分成 k 個子集合時，需要注意 Samples 的 Target 值是否在每一個子集合中的分布相似，因為之後每一個子集合都會被用來當 Testing Set，如果 Training Set 跟 Testing Set 在 Target 的分佈差異過大，可能無法得到好的 Model。在 Model-Selection 中也有一個稱為 StratifiedKFold 的函數，可以做出 "Stratified" 的 k-fold，同學可以參考

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

### Exercise 4

使用 [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)方法重複 Exercise 2 的步驟，請觀察驗證集的選取與 Exercise 2 的差異，請問目標值是否在驗證集當中均勻分配次數

In [ ]:
### Exercise 4




### Exercise 5

針對資料 X, y
1. 使用 [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)方法重複 Exercise 2 的步驟，觀察驗證集的選取，請問目標值是否在驗證集當中均勻分配次數。
2. 使用 [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)方法重複 Exercise 2 的步驟，觀察驗證集的選取。

觀察 KFold 及 StratifiedKFold 選取資料的差異。

In [ ]:
import numpy as np

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8],[9.10],[11,12]])
y = np.array([1, 1, 1, 2, 2, 2])


In [ ]:
# Exercise 5.1






In [ ]:
# Exercise 5.2







今天我們討論了 KFold 及 StratifiedKFold 資料分組的方式，利用分組資料對估計量模型做交叉驗證(cross-validation)，來解決過擬合(overfitting）問題，在今天的練習中，我們將使用這兩種分組的方式，來對估計量模型做交叉驗證，並取得他們的估計準確程度。但是其實 sklearn 有可以直接取得交叉驗證結果的方法，我們下一節再談。

下一節我們要介紹另一個常在 Cross-Validation 使用的函數 : sklearn.model_selection.cross_val_score，這個函數直接使用 Stratified k-fold 來計算 Model 的準確度，可參考官網說明

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
